# Rule: **add_electricity**

**Outputs**

- resources/networks/`base_s_{clusters}_elec.nc`

In [ ]:
######################################## Parameters

### Run
name = ''
prefix = ''

### Network
clusters = 8

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import numpy as np


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_base_s_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_base_s_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (provided by the user, used here to display results at NUTS level. The files must contain at least the columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `base_s_{clusters}_elec.nc`

Load the network and show its components.

In [ ]:
file = f'base_s_{clusters}_elec.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3
link_widths = 1*n.links.p_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, 
       line_widths=line_widths, 
       link_widths=link_widths, 
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.buses`

Place `n.buses` in a dataFrame and show its content.

In [ ]:
bb = n.buses

bb.head()

### Variable: `n.carriers`

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.generators`

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity per carrier? What is the potential capacity per carrier?

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Total_max_capacity=pd.NamedAgg(column='p_nom_max', aggfunc='sum'),
)

#### Summary by resource class

Solar and wind carries may have classes. Show a table with a feature per bus and class.

In [ ]:
#################### Parameters

### Select carrier
carrier = 'onwind'

### Select feature (uncomment one of the following):
feature = 'p_nom'
#feature = 'p_nom_max'



#################### Table
if ('wind' in carrier or 'solar' in carrier):   
    ### Filter
    gg_filtered = gg[gg['carrier']==carrier][['bus', 'p_nom', 'p_nom_max']]
    ### Add class column
    gg_filtered['resource_class'] = gg_filtered.index.to_series().str.split(" ").str[-2]
    ### MW to GW
    gg_filtered[['p_nom', 'p_nom_max']] = gg_filtered[['p_nom', 'p_nom_max']]/1000
    ### Make pivot
    gg_pivot = gg_filtered.pivot(index="bus", columns="resource_class", values=feature)

    styled_table = gg_pivot.style \
        .set_caption(f"Table with {feature} [GW] for {carrier}, according to bus and class.") \
        .format("{:.2f}") \
        .background_gradient(cmap="Blues")  
    
    display(styled_table)

else:
     print(f'Carrier {carrier} is not solar or wind type.. does it contain classes?')



Add a bar plot showing `p_nom` and `p_nom_max`

In [ ]:
#################### Parameters

## bar width
width = 0.3 




#################### Figure

df_sorted = gg_filtered.sort_values(["bus", "resource_class"])

buses = df_sorted["bus"].unique()
resources = df_sorted["resource_class"].unique()

x = np.arange(len(buses))                 
offsets = np.linspace(-width/2.5, width/2.5, len(resources))  


fig, ax = plt.subplots(figsize=(12, 8))

colors = {
    "p_nom_max": "#cfe2f3",
    "p_nom": "#08306b"
}


metrics_order = ["p_nom_max", "p_nom"]

# To control added labels
labels_added = set()

for i, res in enumerate(resources):
    df_res = df_sorted[df_sorted["resource_class"] == res]
    pos = x + offsets[i]
    
    for metric in metrics_order:
        label = metric if metric not in labels_added else None
        ax.bar(
            pos,
            df_res[metric],
            width=width/len(resources),
            color=colors[metric],
            alpha=0.7,
            label=label
        )
        labels_added.add(metric)

ax.set_xticks(x)
ax.grid(axis="y", linestyle="--", alpha=0.6)
ax.set_xlabel("Bus", fontsize=16)
ax.set_ylabel("Capacity [GW]", fontsize=16)
ax.set_xticklabels(buses, fontsize=14)
ax.tick_params(axis="y", labelsize=14)
ax.legend(fontsize=14)

plt.tight_layout()


#### Maps

Plot a map showing a particular feature of a generation carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier
carrier = 'onwind'

### For solar & onwind carriers, select the class number ('all' to aggregate all classes)
resource_class = 'all'

### Select feature (uncomment one of the following):
# feature = 'area' 
feature = 'p_nom'
# feature = 'p_nom_density'
# feature = 'p_nom_max'
# feature = 'p_nom_max_density'
#feature = 'p_nom_max_ratio'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define gdf_regions
if 'off' in carrier:
    gdf_regions = gdf_regions_offshore
else:
    gdf_regions = gdf_regions_onshore


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generators(carrier, n, feature, ax, gdf_regions, resource_class, params['map_network_generators'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 2



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



if 'off' in carrier:
    print('Aggregation at NUTS level for offshore is not possible')
else:
    #################### Figure
    fig_size = [12,6]
    crs = ccrs.PlateCarree()

    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


    ### Define gdf_regions
    gdf_regions = gdf_regions_onshore


    ### Define NUTS file
    if NUTS_level==2:
        gdf_NUTS = gdf_NUTS2
    if NUTS_level==3:
        gdf_NUTS = gdf_NUTS3    


    ### Add map features
    xp.map_add_features(ax, params['map_add_features'])

    ### Add network feature aggregated at NUTS regions
    xp.map_NUTS_generators(carrier, n, f'{feature}_NUTS', ax, gdf_regions, gdf_NUTS, resource_class, params['map_NUTS_generators'], params_local)


#### Costs

What is the capital cost of generators?

In [ ]:
#################### Parameters

### Select carriers
carrier_list = ['onwind', 'solar', 'offwind-float']



#################### Figure
fig_size = [10,5]
fig, ax = plt.subplots(1,1,figsize=fig_size)

# Give a message with the carriers not considered
carrier_all = gg['carrier'].unique().tolist()
carrier_excluded = [carrier for carrier in carrier_all if carrier not in carrier_list]
print(f'Carriers ommitted: {carrier_excluded}')

# Define bins
minimo = 0 # 0.95 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].min()
maximo = 1.05 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].max()
ancho = 1000

bins = np.arange(minimo, maximo, 1000)

# Define colors
tech_colors = n.carriers['color']


for carrier in carrier_list:

    ##### Filter the carrier
    df = gg[gg['carrier']==carrier]


    ##### Only single cost for the carrier
    if df['capital_cost'].round(2).nunique()==1:

        valor = df['capital_cost'].unique()[0]

        ax.axvline(x=valor, label=carrier, color = tech_colors[carrier])

        print(f'Capital cost for {carrier} is: {valor:.2f} EUR/MW·year')

    ##### Different costs for the carrier
    else:
        plt.hist(df['capital_cost'], bins=bins, 
                 edgecolor='none', color = tech_colors[carrier],
                 label=carrier, alpha=1)
        
        valor = df['capital_cost'].mean()
        print(f'Average capital cost for {carrier} is: {valor:.2f} EUR/MW·year')


ax.set_title('capital cost')
ax.set_xlabel('EUR/(MW·year)')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.generators_t[p_max_pu]`

Place `n.generators_t[p_max_pu]` in a dataFrame and show its content.

In [ ]:
ggt_pmaxpu = n.generators_t['p_max_pu']

ggt_pmaxpu.head()

#### Time series

How do the potential generation time series look like?

In [ ]:
#################### Parameters

### Select carrier
carrier = 'onwind'

### For solar & onwind carriers, select the class number ('all' to plot all classes individually)
resource_class = 'all'
    
### Define period
start = '2023-03-01'
end = '2023-03-10'



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)

if ('wind' in carrier or 'solar' in carrier) and resource_class != 'all':
    ggt_pmaxpu.loc[start:end].filter(like=f'{resource_class} {carrier}').plot(ax=ax, alpha=.7, legend=False, linewidth=.5)
else:
    ggt_pmaxpu.loc[start:end].filter(like=carrier).plot(ax=ax, alpha=.7, legend=False, linewidth=.5)

ax.grid(True, linestyle='--', alpha=0.25)
ax.set_ylabel('')

#### Summary by resource class

Solar and wind carries may have classes. Show a table with a feature per bus and class.

In [ ]:
#################### Parameters

### Select carrier
carrier = 'onwind'

### Select feature (uncomment one of the following):
feature = 'CF'



#################### Table
if ('wind' in carrier or 'solar' in carrier):   
    ### Filter
    ggt_pmaxpu_filtered = ggt_pmaxpu.filter(like=carrier, axis=1).mean().to_frame(name='CF')
    ### Add class and bus columns. Use multi-index with 'bus' and 'carrier'    
    split_index = ggt_pmaxpu_filtered.index.to_series().str.rsplit(' ', n=2, expand=True)
    ggt_pmaxpu_filtered['bus'] = split_index[0].values
    ggt_pmaxpu_filtered['resource_class'] = split_index[1].astype(int) # resource_class as integer    
    ### Make pivot
    ggt_pmaxpu_pivot = ggt_pmaxpu_filtered.pivot(index="bus", columns="resource_class", values=feature)

    styled_table = ggt_pmaxpu_pivot.style \
        .set_caption(f"Table with {feature} for {carrier}, according to bus and class.") \
        .format("{:.3f}") \
        .background_gradient(cmap="Blues")
    
    display(styled_table)
else:
    print(f'Carrier {carrier} is not solar or wind type.. does it contain pmax_pu and classes?')


Add a bar plot showing `CF`

In [ ]:
#################### Parameters

## bar width
width = 0.3 




#################### Figure

df_sorted = ggt_pmaxpu_filtered.sort_values(["bus", "resource_class"])

buses = df_sorted["bus"].unique()
resources = df_sorted["resource_class"].unique()

x = np.arange(len(buses))                 
offsets = np.linspace(-width/2.5, width/2.5, len(resources))  


fig, ax = plt.subplots(figsize=(12, 8))

colors = {
    "CF": "#bc91d8",
}


metrics_order = ["CF"]

# To control added labels
labels_added = set()

for i, res in enumerate(resources):
    df_res = df_sorted[df_sorted["resource_class"] == res]
    pos = x + offsets[i]
    
    for metric in metrics_order:
        label = metric if metric not in labels_added else None
        ax.bar(
            pos,
            df_res[metric],
            width=width/len(resources),
            color=colors[metric],
            alpha=0.7,
            label=label
        )
        labels_added.add(metric)

ax.set_xticks(x)
ax.grid(axis="y", linestyle="--", alpha=0.6)
ax.set_xlabel("Bus", fontsize=16)
ax.set_ylabel("CF", fontsize=16)
ax.set_xticklabels(buses, fontsize=14)
ax.tick_params(axis="y", labelsize=14)
ax.legend(fontsize=14)

plt.tight_layout()


#### Maps

Plot a map showing a particular feature of the potential generation for a carrier at each region.

In [ ]:
#################### Parameters

##### Select carrier:
carrier = 'onwind'

### For solar & onwind carriers, select the class number (note that here, 'all' is not valid)
resource_class = 0

##### Select feature (uncomment one of the following):
feature = 'CF' 



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define gdf_regions
if 'off' in carrier:
    gdf_regions = gdf_regions_offshore
else:
    gdf_regions = gdf_regions_onshore


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generatorst_pmaxpu(carrier, n, feature, ax, gdf_regions, params['map_network_generatorst_pmaxpu'], params_local, resource_class)

### Variable: `n.lines`

Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the distribution of line lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(ln['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

How is the relationship between line capital costs and line length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR/MW')
ax.grid(True, linestyle='--', alpha=0.5)


ratio = ln['capital_cost']/ln['length']

print(f'The ratio values of capital cost vs. length  are {ratio.round(2).unique()} EUR/(MW·km)')

### Variable: `n.links`

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

Show the carriers associated with links.

In [ ]:
lk['carrier'].drop_duplicates().reset_index(drop=True)

#### Link type: DC

Place DC links in a dataFrame and show its content.

In [ ]:
lk_DC = lk.loc[lk['carrier']=='DC']

lk_DC.head()

How is the distribution of DC link lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(lk_DC['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

How is the relationship between DC link capital costs and link length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


lk_DC.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)


ratio = lk_DC['capital_cost']/lk_DC['length']

print(f'The ratio values of capital cost vs. length  are {ratio.round(2).unique()} EUR/km')

#### Link types: H2 electrolysis, H2 fuell cell, battery charger, battery discharger

**NOTE**: Links represent different elements in the network. The aggregated capacity per carrier may need to be divided by the efficiency to properly reflect the capacity. For example, for CCGT, the link represents the conversion from gas to electricity, and the associated capacity refers to the input node. However, the CCGT capacity refers to the power capacity (output node).

In the following, **"_e"** is added to express "electric" (power output, the efficiency is included).

##### Maps

Plot a map showing a particular feature of a link carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier (uncomment one of the following)
carrier = 'H2 electrolysis'
# carrier = 'H2 fuel cell'
# carrier = 'battery charger'
# carrier = 'battery discharger'

### Select feature (uncomment one of the following):
# feature = 'area' 
feature = 'p_nom_e'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_links(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_links'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_links(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_links'], params_local)

### Variable: `n.loads_t[p_set]`

Place `n.loads_t[p_set]` in a dataFrame and show its content.

In [ ]:
lot_pset = n.loads_t['p_set']

lot_pset.head()

#### Time series

How do the load time series look like?

In [ ]:
#################### Parameters
start = '2023-03-01'
end = '2023-03-10'



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)

lot_pset.loc[start:end].plot(ax=ax, alpha=.4, legend=False, linewidth=.5)

ax.grid(True, linestyle='--', alpha=1)
ax.set_ylabel('MW')

#### Maps

Plot a map showing a particular feature of the load at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'annual_load'
feature = 'annual_load_density'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_loadst_pset(n, feature, ax, gdf_regions_onshore, params['map_network_loadst_pset'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 2



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_loadst_pset(n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_loadst_pset'], params_local)

### Variable: `n.storage_units`

Place `n.storage_units` in a dataFrame and show its content.

In [ ]:
su = n.storage_units

su.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have storage units for each carrier? How many of them have zero capacity?

In [ ]:
su.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Maps

Plot a map showing a particular feature of a storage unit carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier
carrier = 'PHS'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom'
# feature = 'p_nom_density'
feature = 'max_hours'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_storage_units(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_storage_units'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 2



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_storage_units(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_storage_units'], params_local)

#### Maximum hours

What is the relation between installed capacity and max. hours, for each carrier?

In [ ]:
#################### Figure
fig_size = [8,4]
fig, ax = plt.subplots(figsize=fig_size)

tech_colors = n.carriers['color']

for carrier, group in su.groupby('carrier'):
    ax.scatter(group['p_nom'], group['max_hours'], label=carrier, color=tech_colors[carrier], alpha=0.7)

ax.set_xlabel('Installed capacity [MW]')
ax.set_ylabel('Max. hours')
ax.set_title('Storage units')
ax.grid(True, linestyle='--', alpha=0.5)
ax.legend()

### Variable: `n.stores`

Place `n.stores` in a dataFrame and show its content.

In [ ]:
st = n.stores

st.head()